In [5]:
import json
from pathlib import Path
from slugify import slugify
import shutil
import requests
import yaml
from datetime import datetime

from bs4 import BeautifulSoup
import html2text

In [6]:
with open("db1033256-1.json", "r") as f:
    data = json.load(f)

In [7]:
for index, item in enumerate(data[2:]):
    #print(index + 2, item["name"])
    pass
    
print(data[5]["name"])

astropics = data[5]

Astropic


In [8]:
astropics["data"][0]

{'ID': '2',
 'ClassName': 'Astropic',
 'LastEdited': '2018-12-31 14:41:23',
 'Created': '2016-06-05 12:53:51',
 'Title': 'Sternhimmel - Bosenberg',
 'Autor': 'Mehmet Ergün',
 'Kamera': 'Nikon D750 DSLR',
 'ISO': '3200',
 'Blende': '2,8',
 'Belichtung': '20',
 'AufnahmeOrt': 'Bosenberg Bosenheim',
 'AufnahmeDatum': '2016-03-03',
 'Teleskop': None,
 'DatenZumObjekt': 'Sternhimmel über Bad Kreuznach (Ortsteil Bosenheim)',
 'BildID': '285',
 'AufnahmeZeit': '00:00:00',
 'Objektiv': 'Tamron 15-30mm',
 'Nachbearbeitung': 'Adobe Lightroom',
 'TeleskopExtras': None,
 'KameraFilter': None,
 'Frames': None,
 'CategoryID': '2',
 'CreatedByID': '9',
 'EditedByID': '9'}

In [9]:
def get_download_link(data, astropic):
    results = find_image(data, astropic)
    assert len(results) == 1
    image_object = results[0]
    image_file_location = "http://sternwarte-kreuznach.de/assets/" + image_object["FileFilename"]
    return image_file_location

In [10]:
def download_file(file_location, file_destination):
    if not Path(file_destination).is_file():
        r = requests.get(file_location, stream=True)

        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            # Open a local file with wb ( write binary ) permission.
            with open(file_destination, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

            downloaded_image = True

        else:
            print('Image could not be retreived!')    

In [11]:
def find_image(data, astropic):
    image_id = int(astropic["BildID"])
    file_database = data[43]["data"]

    result = [entry for entry in file_database if int(entry["ID"]) == image_id]

    assert len(result) <= 1
    return result

In [12]:
def write_file(folder, yaml_dict, content):
    with open(folder.joinpath("index.md"), "w") as md:
        markdown_output = [
            "---\n",
            yaml.dump(yaml_dict, allow_unicode=True),
            "---\n",
            content
        ]
        md.writelines(markdown_output)

In [41]:
def get_content(file_name: str):
    print("Looking for something that ends with '{}'".format(file_name[-20:]))
    links = ["https://sternwarte-kreuznach.de/galerie?s={}".format(i*12) for i in range(8)]
    for link in links:
        print("Trying link {}".format(link))
        r = requests.get(link)
        soup = BeautifulSoup(r.content, "html.parser")
        
        row = soup.find_all("div", {"class": "row"})[0]
        all_as = row.find_all("a")
        for a_index in all_as:
            image_name = a_index.attrs["href"].split("/")[-1]
            
            print(image_name)
            if file_name.endswith(image_name):
                soup2 = BeautifulSoup(a_index.attrs['data-caption'])
                entry = html2text.html2text(soup2.decode(), bodywidth=0)
                return entry        

In [42]:
get_content("http://sternwarte-kreuznach.de/assets/astropics/Bild18-klein.jpg")

Looking for something that ends with 'ics/Bild18-klein.jpg'
Trying link https://sternwarte-kreuznach.de/galerie?s=0
Adobe_20210312_210841.jpg
C2020-F3_7.jpg
Orion-Tripletsmall.jpg
Adobe_20191201_075037.jpg
20191027_081536.jpg
SH2-132_Loewennebel_Bicolor.jpeg
IC1396-final.jpg
NGC6995_Eastern-Veil-Nebula.jpg
M16_Adlernebel.jpeg
LRM_EXPORT_22302740599701_20190624_073009741.jpeg
0001_lapl4_ap186_Drizzle15_3_RGB1.jpg
M81-Bodes-Galaxie_14-2.jpg
Trying link https://sternwarte-kreuznach.de/galerie?s=12
MG_9617.jpg
Orion.jpg
final_MW_TF.jpg
M31_nikon1.jpg
ANTARES_BEST.jpg
pferd_ha1.jpg
orion_final.jpg
350_Antares.jpg
test2_final.jpg
Final.jpg
M33_8-Bearbeitet-2.jpg
M27_10-Bearbeitet-Bearbeitet.jpg
Trying link https://sternwarte-kreuznach.de/galerie?s=24
IC-434-Bearbeitet-2.jpg
41P-smal.jpg
20170316-MG-8835-smal.jpg
11-1-von-1-15.jpg
11-1-von-1-24.jpg
11-1-von-1-18.jpg
11-1-von-1-13-stitch.jpg
11-1-von-1-17.jpg
M42-final-01.jpg
pferdekopf-dez04-final.jpg
M81-82-final.jpg
orion-final-best.jpg
Try

'Autor| Mehmet Ergün  \n---|---  \nKamera| Nikon D750 DSLR  \nObjektiv| Tamron 15-30mm  \nNachbearbeitung| Lightroom  \nISO| 3200  \nBlende| f/2,8  \nBelichtung| 20 sec.  \nAufnahmeort| Bosenberg Bosenheim  \nAufnahmedatum| 3\\. März 2016, 00:00 Uhr  \nDaten zum Objekt| Sternhimmel über Bad Kreuznach (Ortsteil Bosenheim)  \nKategorie| Landschaft\n'

In [43]:
used_paths = []
for astropic in astropics["data"]:
    friendly_name = slugify(astropic["Title"], max_length=255, replacements=[
        ['Ä', 'AE'], ['ä', 'ae'],
        ['Ö', 'OE'], ['ö', 'oe'],
        ['Ü', 'UE'], ['ü', 'ue'],
        ['ß', 'ss'],
    ])
    
    year = astropic["Created"][:4]    
    folder = Path("..", "content", "galerie", year, friendly_name)
    
    index = 1    
    while str(folder) in used_paths:
        folder = Path("..", "content", "galerie", year, "{}-{}".format(friendly_name, index))
        index += 1
    
    used_paths.append(str(folder))
    download_link = get_download_link(data, astropic)
    file_extension = download_link.split(".")[-1]
    
    file_name = folder.name    
    folder.mkdir(exist_ok=True, parents=True)
    file_name = file_name + "." + file_extension
    
    target_file = folder.joinpath(file_name) 
    print(download_link)
    
    download_file(download_link, target_file)
    
    yaml_dict = dict()
    yaml_dict["title"] = astropic["Title"]
    yaml_dict["images"] = [target_file.name]
    yaml_dict["date"] = datetime.strptime(astropic["Created"], '%Y-%m-%d %H:%M:%S')
    write_file(folder, yaml_dict, get_content(download_link))    

http://sternwarte-kreuznach.de/assets/astropics/Bild18-klein.jpg
Looking for something that ends with 'ics/Bild18-klein.jpg'
Trying link https://sternwarte-kreuznach.de/galerie?s=0
Adobe_20210312_210841.jpg
C2020-F3_7.jpg
Orion-Tripletsmall.jpg
Adobe_20191201_075037.jpg
20191027_081536.jpg
SH2-132_Loewennebel_Bicolor.jpeg
IC1396-final.jpg
NGC6995_Eastern-Veil-Nebula.jpg
M16_Adlernebel.jpeg
LRM_EXPORT_22302740599701_20190624_073009741.jpeg
0001_lapl4_ap186_Drizzle15_3_RGB1.jpg
M81-Bodes-Galaxie_14-2.jpg
Trying link https://sternwarte-kreuznach.de/galerie?s=12
MG_9617.jpg
Orion.jpg
final_MW_TF.jpg
M31_nikon1.jpg
ANTARES_BEST.jpg
pferd_ha1.jpg
orion_final.jpg
350_Antares.jpg
test2_final.jpg
Final.jpg
M33_8-Bearbeitet-2.jpg
M27_10-Bearbeitet-Bearbeitet.jpg
Trying link https://sternwarte-kreuznach.de/galerie?s=24
IC-434-Bearbeitet-2.jpg
41P-smal.jpg
20170316-MG-8835-smal.jpg
11-1-von-1-15.jpg
11-1-von-1-24.jpg
11-1-von-1-18.jpg
11-1-von-1-13-stitch.jpg
11-1-von-1-17.jpg
M42-final-01.jpg
pfe